# Correlation Power Analysis Example

- Setup for example AES
- Uses Picoscope

## Data Acquisition

In [ ]:
# capture_picoscope
# requires pynqserver to run on Pynq
# requires FOBOS Shield
# uses Picoscope
import os
import sys
import shutil
import json
import numpy as np
from foboslib import projmgr
from foboslib.ctrl import pynqctrl
from foboslib.ctrl.fobosctrl import FOBOSCtrl
from foboslib.fobosTVGen import FobosTVGen
#from foboslib.dut import cw305dut as DUT
from foboslib.dut import nexys4dut as DUT
from foboslib.scope.picoscope3000 import Picoscope

#######################################################
# Acquisition Configuration
acqConf = {}
# Connection to Control Board
acqConf['ip'] = '192.168.10.93'
acqConf['port'] = 9995
# File Names and Locations
acqConf['workspace']       = "fobosworkspace"   # Folder to store projects
acqConf['projectName']     = "aes_pico"              # Name of this project. Files will be in workspace/projectName/
acqConf['keyFile']         = "key.txt"          # key, must be provided in workspace/projectName/ directory
acqConf['plainFile']       = "plaintext.txt"    # plaintext in hex, auto-generated
acqConf['dinFile']         = "dinFile.txt"      # test vectors in FOBOS format incl. plaintext and key, auto-generated
acqConf['cipherFile']      = "ciphertext.txt"   # observed ciphertext, result from applying test vectors
acqConf['traceFile']       = "powerTraces.npy"  # Name of file for recording power traces
# Cipher Specific Information
acqConf['bitFile']         = "aes_nexys4.bit"    # Bitstream for programming the DUT, same directory as this notebook
acqConf['blockSize']       = 16                 # size of a block of plaintext in bytes
acqConf['cipherSize']      = 16                 # size of a block of cyphertext in bytes
acqConf['keySize']         = 16                 # size of the key in bytes
# Acquistion Configuration
acqConf['traceNum']        = 10              # number of traces to run
acqConf['samplingFreq']    = 50                # sampling frequency of the Oscilloscope in Msps
acqConf['DUTClk']          = 0.5                # clock frequency of the DUT in MHz [range: 1 - 100]
acqConf['samplesPerTrace'] = 4000               # number of ADC samples in one trace
# PicoScope Parameters
acqConf['Channel']         = 'CHANNEL_A'        # CHANNEL_A or CHANNEL_B
acqConf['Coupling']        = 'AC'               # AC or DC
acqConf['Range']           = 200                # Range in mV
acqConf['SamplingInterval'] = 1/(acqConf['samplingFreq']/1000) # or set below
#acqConf['SamplingInterval']= 8                  # in ns = 1/sample frequency
acqConf['Trigger_Chan']    = 'EXTERNAL'         # CHANNEL_A, CHANNEL_B, or EXTERNAL
acqConf['Trigger_Edge']    = 'RISING_EDGE'      # RISING_EDGE only as FOBOS trigger signals start with that
acqConf['Trigger_Threashold']= 400              # ??? on External?

In [ ]:
# Configure project directories
pm = projmgr.ProjectManager()
pm.setWorkSpaceDir(acqConf['workspace'])
pm.setProjName(acqConf['projectName'])
projDir = pm.getProjDir()

In [ ]:
# create key file in tje workspace directory
KEY = "01 23 45 67 89 ab cd ef 00 11 22 33 44 55 66 77"
keyFileName = os.path.join(projDir, acqConf['keyFile'])
keyFile = open(keyFileName, "w")
keyFile.write(KEY )
keyFile.close()

In [ ]:
# Generate test vectors
tvGen = FobosTVGen(traceNum=acqConf['traceNum'],
                   blockSize=acqConf['blockSize'],
                   keySize=acqConf['keySize'],
                   cipherSize=acqConf['cipherSize'],
                   dinFile= os.path.join(projDir, acqConf['dinFile']),
                   plaintextFile=os.path.join(projDir, acqConf['plainFile']),
                   keyFile=os.path.join(projDir, acqConf['keyFile'])
                   )
tvGen.generateTVs()

In [ ]:
# Create capture directory
captureDir = pm.getCaptureDir()
# Open input files
plainFileName = os.path.join(projDir, acqConf['plainFile'])
tvFileName = os.path.join(projDir, acqConf['dinFile'])
tvFile = open(tvFileName, "r")
# Open output files
cipherFileName = os.path.join(captureDir, acqConf['cipherFile'])
traceFileName = os.path.join(captureDir, acqConf['traceFile'])
cipherFile = open(cipherFileName, "w")
traceFile = open(traceFileName, "a+b")
# save input files
shutil.copy(tvFileName, captureDir)
shutil.copy(plainFileName, captureDir)
# save config to a file
configFile = open(os.path.join(captureDir, 'acquisitionConfig.json'), "w")
configFile.write(json.dumps(acqConf, indent=4))
configFile.close()

In [ ]:
### Connecting to the Pynq Board
# lock file is: /tmp/fobos.lock
# delete the lock file manually if its stale
ctrl = pynqctrl.PYNQCtrl(acqConf['ip'], acqConf['port'])

In [ ]:
# Apply settings to the Pynq Board
ctrl.setDUTClk(acqConf['DUTClk'])                 # setting DUT clock resets all modules using this clock
ctrl.setDUTInterface(FOBOSCtrl.INTERFACE_4BIT)    # Using Target connector -> INTERFACE_4BIT
ctrl.setDUT(FOBOSCtrl.NEXYS4)                     # Adjust Target connector pinout for DUT
ctrl.setTriggerMode(FOBOSCtrl.TRG_FULL)           # Trigger is active during DUT operation
ctrl.setOutLen(acqConf['cipherSize'])
#ctrl.setSamplingFrequency(acqConf['samplingFreq'])
#ctrl.setSamplesPerTrace(acqConf['samplesPerTrace'])
#ctrl.setADCGain(acqConf['ADCGain'])
#ctrl.setADCHiLo(acqConf['ADCHiLo'])

In [ ]:
# Setup Picoscope
scope = Picoscope(postTriggerSamples=acqConf['samplesPerTrace'])
scope.setChannel(channelName=acqConf['Channel'], coupling=acqConf['Coupling'], rangemv=acqConf['Range'])
scope.setSamplingInterval(samplingIntervalns=acqConf['SamplingInterval'])
scope.setTrigger(channelName=acqConf['Trigger_Chan'], direction=acqConf['Trigger_Edge'], 
                 thresholdmv=acqConf['Trigger_Threashold'])

In [ ]:
# Program the DUT
dut = DUT.DUT()
dut.setBitFile(acqConf['bitFile'])
dut.program()

In [ ]:
# Run the data acquisition
print('Processing test vectors ...')
traceNum = 0
while traceNum < acqConf['traceNum']:
    scope.arm()
    data = tvFile.readline()
    status, result = ctrl.processData(data, acqConf['cipherSize'])
    cipherFile.write(result + "\n")
    trace = scope.readTrace()
    np.save(traceFile, trace)
    if traceNum % 100 == 0:
        sys.stdout.write('Progress:' + "{:.2f}".format(traceNum/acqConf['traceNum']*100) + '%\r')
        sys.stdout.flush()
    traceNum += 1
print('Data acquisition complete.')
scope.closeConnection()
# release and reset control board.
ctrl.disconnect() 
# close all files
traceFile.close()
cipherFile.close()
tvFile.close()

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
fig.patch.set_facecolor('white')
plt.rcParams.update({'font.size': 18})
traceFile = open(traceFileName, "r+b")
maxtrace = 100
plt.figure(figsize=(10,8))
plt.xlabel('Sample')
plt.ylabel('Amplitude')
plt.title('Captured Traces')
for i in range(min(maxtrace, acqConf['traceNum'])):
    trace = np.load(traceFile)
    plt.plot(trace)

plt.savefig(os.path.join(captureDir, 'traces.png'),facecolor=fig.get_facecolor())
# plt.close()
traceFile.close()

In [ ]:
print(result)

## Correlation Power Analysis

In [ ]:
import foboslib.cpa as cpa
import foboslib.traceset as traceset
import foboslib.powermodels.AESFirstLast as powermodel #code to calculate hypothetical power
import foboslib.powermodels.utils as powermodelUtils

In [ ]:
# Configure project directories
WORKSPACE = os.path.dirname(captureDir)           # or specify other directory if needed
ATTEMPT_DIR = os.path.basename(captureDir)        # or specify other directory if needed, e.g., "attempt-24"

In [ ]:
# Create analysis project directory
pm = projmgr.ProjectManager()
pm.setWorkSpaceDir(WORKSPACE)
pm.setProjName(ATTEMPT_DIR)
projDir = pm.getProjDir()
analysisDir = pm.getAnalysisDir()

In [ ]:
# read acquisition settings
cpaConf = {}
configFile = open(os.path.join(projDir, 'acquisitionConfig.json'))
acqConf = json.load(configFile)
configFile.close()
print(f'Acquisition config = {acqConf}')
cpaConf['traceNum'] = acqConf['traceNum']

# Configure file names
HYPO_FILE = os.path.join(projDir, "hypotheticalPower.npy")
TRACES_FILE = os.path.join(projDir, acqConf['traceFile'])
PLAIN_TEXT = os.path.join(projDir, acqConf['plainFile'])
CIPHER_TEXT = os.path.join(projDir, acqConf['cipherFile'])

In [ ]:
# Create hypothetical power model

#if os.path.isfile(HYPO_FILE):
    # already exists. load it
#    hypotheticalPower = powermodelUtils.loadHypoPower(HYPO_FILE)
#else:
    # not there. generate it
hypotheticalPower = powermodel.getHypotheticalPower(PLAIN_TEXT, CIPHER_TEXT, cpaConf['traceNum'])
powermodelUtils.saveHypoPower(hypotheticalPower, HYPO_FILE)
## end if
powermodelUtils.showHypoPower(hypotheticalPower, plainFile=PLAIN_TEXT, cipherFile=CIPHER_TEXT)

In [ ]:
# load traces from file and crop them.
cpaConf['cropStart'] = 800
cpaConf['cropEnd'] = 820
traceSet = traceset.TraceSet(traceNum=cpaConf['traceNum'],
                            fileName=TRACES_FILE,
                            cropStart=cpaConf['cropStart'],
                            cropEnd=cpaConf['cropEnd'])

measuredPower = traceSet.traces
print(f'The shape of the traces matrix is {measuredPower.shape}')


maxtrace = 100
fig = plt.figure()
fig.patch.set_facecolor('white')
plt.rcParams.update({'font.size': 18})
traceFile = open(traceFileName, "r+b")
maxtrace = 100
plt.figure(figsize=(10,8))
plt.xlabel('Sample')
plt.ylabel('Amplitdue')
plt.title('Cropped Traces')
for i in range(min(maxtrace, cpaConf['traceNum'])):
    plt.plot(measuredPower[i])
plt.savefig(os.path.join(analysisDir, 'cropped_traces.png'),facecolor=fig.get_facecolor())

In [ ]:
# Configure the attack
numKeys = 16              # number of subkey bytes to attack
cpaConf['plot'] = True    # generate plots, [True|False]
#####################
# Run the attack
cpaAttacker = cpa.CPA()
C = cpaAttacker.doCPA(measuredPower=measuredPower,
                      hypotheticalPower=hypotheticalPower,
                      numTraces= cpaConf['traceNum'],
                      analysisDir=analysisDir,
                      MTDStride= 100,
                      numKeys = numKeys,
                      plot= cpaConf['plot'], # enable/disable plots
                      plotSize=(10,8),
                      plotFontSize=18
                      )

In [ ]:
# Writing analysis configuration to file.
configFile = open(os.path.join(analysisDir, 'analysisConfig.json'), "w")
configFile.write(json.dumps(cpaConf, indent=4))
configFile.close()

In [ ]:
scope.closeConnection()